In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer

from tqdm.notebook import tqdm
from datetime import datetime
import multiprocessing
import logging

cores = multiprocessing.cpu_count()
print(f'cores: {cores}')

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

cores: 4


In [2]:
path_data = Path('../../data/corpora/clean_tutby_126784.csv')
# path_data = Path('../../data/corpora/clean_tutby_126784_header.csv')

path_w2v = Path('../../data/model/w2v/model_w2v_clean_tutby_126784.model')
# path_w2v = Path('../../data/model/w2v/model_w2v_clean_tutby_126784_header.model')

path_emb = Path('../../data/emb/emb_clean_tutby_126784_w2v_idf.npy')
# path_emb = Path('../../data/emb/emb_clean_tutby_126784_header_w2v_idf.npy')

column_name = 'document'
# column_name = 'header'

In [3]:
%%time

data = pd.read_csv(path_data)

documents = data[column_name]
documents = documents.fillna('')

documents = documents.str.split()
documents = documents.tolist()

print(data.shape)
data.head(5)

(126784, 6)
Wall time: 14.8 s


,url,label,header,date,document,tags
0,https://news.tut.by/550306.html,Футбол,"Тренер ""Шахтера"": Оправдываться не хочу. Все в...",2017-07-06T21:35:00+03:00,главный тренер солигорский шахтер олег кубарев...,['футбол']
1,https://news.tut.by/550307.html,Общество,"""Зацветет"" ли каменная роза на ул. Комсомольск...",2017-07-07T09:25:00+03:00,план восстановление рисунок пока художник илья...,"['архитектура', 'живопись', 'ЖКХ']"
2,https://news.tut.by/550308.html,Общество,Фотофакт. Скамейка в виде пожарной машины появ...,2017-07-07T09:27:00+03:00,областной управление мчс день пожарный служба ...,['министерства']
3,https://news.tut.by/550309.html,Футбол,Станислав Драгун дебютировал за БАТЭ в матче с...,2017-07-06T22:11:00+03:00,чемпион беларусь бате воспользоваться пауза че...,"['футбол', 'БАТЭ']"
4,https://news.tut.by/550310.html,В мире,Генпрокурор Украины пообещал открыть уголовное...,2017-07-06T22:28:00+03:00,генпрокуратура украина открывать уголовный про...,"['Ситуация в Украине', 'государственные перево..."


In [4]:
%%time
print(datetime.now())

params_word2vec = {
    'size': 300,
    'window': 5,
    'min_count': 20,
    'workers': cores,
    'sg': 1,
    'negative': 5,
    'sample': 1e-5,
    'iter': 150,
}

model = Word2Vec(sentences=documents, **params_word2vec)

model.save(str(path_w2v))
model = Word2Vec.load(str(path_w2v))

print(len(model.wv.vocab))

2020-11-15 12:29:26,237 : INFO : collecting all words and their counts
2020-11-15 12:29:26,239 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


2020-11-15 12:29:26.175985


2020-11-15 12:29:28,191 : INFO : PROGRESS: at sentence #10000, processed 2466256 words, keeping 99885 word types
2020-11-15 12:29:29,383 : INFO : PROGRESS: at sentence #20000, processed 5035139 words, keeping 143097 word types
2020-11-15 12:29:30,227 : INFO : PROGRESS: at sentence #30000, processed 7623391 words, keeping 176652 word types
2020-11-15 12:29:30,907 : INFO : PROGRESS: at sentence #40000, processed 10215111 words, keeping 205336 word types
2020-11-15 12:29:31,509 : INFO : PROGRESS: at sentence #50000, processed 12787803 words, keeping 231745 word types
2020-11-15 12:29:32,140 : INFO : PROGRESS: at sentence #60000, processed 15423913 words, keeping 256913 word types
2020-11-15 12:29:32,993 : INFO : PROGRESS: at sentence #70000, processed 18031546 words, keeping 279946 word types
2020-11-15 12:29:33,914 : INFO : PROGRESS: at sentence #80000, processed 20691805 words, keeping 302790 word types
2020-11-15 12:29:34,582 : INFO : PROGRESS: at sentence #90000, processed 23266690 wo

2020-11-15 12:30:47,022 : INFO : EPOCH 1 - PROGRESS: at 58.14% examples, 136697 words/s, in_qsize 8, out_qsize 0
2020-11-15 12:30:48,084 : INFO : EPOCH 1 - PROGRESS: at 59.24% examples, 136647 words/s, in_qsize 7, out_qsize 0
2020-11-15 12:30:49,084 : INFO : EPOCH 1 - PROGRESS: at 60.27% examples, 136717 words/s, in_qsize 7, out_qsize 0
2020-11-15 12:30:50,089 : INFO : EPOCH 1 - PROGRESS: at 61.34% examples, 136761 words/s, in_qsize 6, out_qsize 1
2020-11-15 12:30:51,106 : INFO : EPOCH 1 - PROGRESS: at 62.37% examples, 136724 words/s, in_qsize 7, out_qsize 0
2020-11-15 12:30:52,111 : INFO : EPOCH 1 - PROGRESS: at 63.54% examples, 136792 words/s, in_qsize 7, out_qsize 0
2020-11-15 12:30:53,153 : INFO : EPOCH 1 - PROGRESS: at 64.78% examples, 136926 words/s, in_qsize 8, out_qsize 0
2020-11-15 12:30:54,181 : INFO : EPOCH 1 - PROGRESS: at 65.93% examples, 136852 words/s, in_qsize 8, out_qsize 0
2020-11-15 12:30:55,191 : INFO : EPOCH 1 - PROGRESS: at 66.99% examples, 136796 words/s, in_qsiz

2020-11-15 12:31:58,052 : INFO : EPOCH 2 - PROGRESS: at 24.99% examples, 123199 words/s, in_qsize 7, out_qsize 0
2020-11-15 12:31:59,053 : INFO : EPOCH 2 - PROGRESS: at 25.93% examples, 123167 words/s, in_qsize 7, out_qsize 0
2020-11-15 12:32:00,112 : INFO : EPOCH 2 - PROGRESS: at 26.79% examples, 122504 words/s, in_qsize 8, out_qsize 0
2020-11-15 12:32:01,204 : INFO : EPOCH 2 - PROGRESS: at 27.56% examples, 121583 words/s, in_qsize 7, out_qsize 0
2020-11-15 12:32:02,275 : INFO : EPOCH 2 - PROGRESS: at 28.46% examples, 121276 words/s, in_qsize 7, out_qsize 0
2020-11-15 12:32:03,326 : INFO : EPOCH 2 - PROGRESS: at 29.48% examples, 121247 words/s, in_qsize 8, out_qsize 1
2020-11-15 12:32:04,348 : INFO : EPOCH 2 - PROGRESS: at 30.53% examples, 121419 words/s, in_qsize 7, out_qsize 0
2020-11-15 12:32:05,395 : INFO : EPOCH 2 - PROGRESS: at 31.41% examples, 120793 words/s, in_qsize 8, out_qsize 0
2020-11-15 12:32:06,401 : INFO : EPOCH 2 - PROGRESS: at 32.57% examples, 121570 words/s, in_qsiz

...

2020-11-15 16:20:28,947 : INFO : EPOCH 148 - PROGRESS: at 5.94% examples, 119073 words/s, in_qsize 8, out_qsize 0
2020-11-15 16:20:29,972 : INFO : EPOCH 148 - PROGRESS: at 7.20% examples, 123279 words/s, in_qsize 7, out_qsize 0
2020-11-15 16:20:31,042 : INFO : EPOCH 148 - PROGRESS: at 8.36% examples, 125666 words/s, in_qsize 7, out_qsize 0
2020-11-15 16:20:32,069 : INFO : EPOCH 148 - PROGRESS: at 9.32% examples, 124533 words/s, in_qsize 7, out_qsize 0
2020-11-15 16:20:33,070 : INFO : EPOCH 148 - PROGRESS: at 10.28% examples, 124337 words/s, in_qsize 7, out_qsize 0
2020-11-15 16:20:34,077 : INFO : EPOCH 148 - PROGRESS: at 11.24% examples, 124348 words/s, in_qsize 8, out_qsize 0
2020-11-15 16:20:35,093 : INFO : EPOCH 148 - PROGRESS: at 12.27% examples, 124271 words/s, in_qsize 8, out_qsize 0
2020-11-15 16:20:36,111 : INFO : EPOCH 148 - PROGRESS: at 13.26% examples, 124815 words/s, in_qsize 6, out_qsize 1
2020-11-15 16:20:37,114 : INFO : EPOCH 148 - PROGRESS: at 14.45% examples, 126177 wo

2020-11-15 16:21:42,566 : INFO : EPOCH 148 - PROGRESS: at 88.44% examples, 143123 words/s, in_qsize 7, out_qsize 0
2020-11-15 16:21:43,601 : INFO : EPOCH 148 - PROGRESS: at 89.61% examples, 143188 words/s, in_qsize 8, out_qsize 0
2020-11-15 16:21:44,605 : INFO : EPOCH 148 - PROGRESS: at 90.68% examples, 143236 words/s, in_qsize 6, out_qsize 1
2020-11-15 16:21:45,636 : INFO : EPOCH 148 - PROGRESS: at 91.93% examples, 143335 words/s, in_qsize 7, out_qsize 0
2020-11-15 16:21:46,639 : INFO : EPOCH 148 - PROGRESS: at 93.12% examples, 143441 words/s, in_qsize 7, out_qsize 0
2020-11-15 16:21:47,642 : INFO : EPOCH 148 - PROGRESS: at 94.22% examples, 143481 words/s, in_qsize 6, out_qsize 0
2020-11-15 16:21:48,664 : INFO : EPOCH 148 - PROGRESS: at 95.24% examples, 143393 words/s, in_qsize 8, out_qsize 0
2020-11-15 16:21:49,675 : INFO : EPOCH 148 - PROGRESS: at 96.34% examples, 143436 words/s, in_qsize 7, out_qsize 0
2020-11-15 16:21:50,701 : INFO : EPOCH 148 - PROGRESS: at 97.59% examples, 14354

2020-11-15 16:22:51,513 : INFO : EPOCH 149 - PROGRESS: at 65.14% examples, 141805 words/s, in_qsize 6, out_qsize 1
2020-11-15 16:22:52,549 : INFO : EPOCH 149 - PROGRESS: at 66.44% examples, 141942 words/s, in_qsize 8, out_qsize 0
2020-11-15 16:22:53,554 : INFO : EPOCH 149 - PROGRESS: at 67.60% examples, 142051 words/s, in_qsize 8, out_qsize 0
2020-11-15 16:22:54,569 : INFO : EPOCH 149 - PROGRESS: at 68.73% examples, 142149 words/s, in_qsize 8, out_qsize 1
2020-11-15 16:22:55,569 : INFO : EPOCH 149 - PROGRESS: at 69.87% examples, 142294 words/s, in_qsize 8, out_qsize 0
2020-11-15 16:22:56,590 : INFO : EPOCH 149 - PROGRESS: at 71.05% examples, 142386 words/s, in_qsize 7, out_qsize 0
2020-11-15 16:22:57,629 : INFO : EPOCH 149 - PROGRESS: at 72.22% examples, 142444 words/s, in_qsize 7, out_qsize 0
2020-11-15 16:22:58,652 : INFO : EPOCH 149 - PROGRESS: at 73.35% examples, 142515 words/s, in_qsize 8, out_qsize 0
2020-11-15 16:22:59,652 : INFO : EPOCH 149 - PROGRESS: at 74.43% examples, 14261

2020-11-15 16:24:00,773 : INFO : EPOCH 150 - PROGRESS: at 44.12% examples, 147099 words/s, in_qsize 8, out_qsize 0
2020-11-15 16:24:01,811 : INFO : EPOCH 150 - PROGRESS: at 45.27% examples, 147057 words/s, in_qsize 8, out_qsize 0
2020-11-15 16:24:02,869 : INFO : EPOCH 150 - PROGRESS: at 46.44% examples, 147119 words/s, in_qsize 8, out_qsize 0
2020-11-15 16:24:03,882 : INFO : EPOCH 150 - PROGRESS: at 47.63% examples, 147203 words/s, in_qsize 8, out_qsize 1
2020-11-15 16:24:04,898 : INFO : EPOCH 150 - PROGRESS: at 48.78% examples, 147254 words/s, in_qsize 7, out_qsize 1
2020-11-15 16:24:05,905 : INFO : EPOCH 150 - PROGRESS: at 49.97% examples, 147317 words/s, in_qsize 6, out_qsize 1
2020-11-15 16:24:06,921 : INFO : EPOCH 150 - PROGRESS: at 51.11% examples, 147280 words/s, in_qsize 7, out_qsize 0
2020-11-15 16:24:07,940 : INFO : EPOCH 150 - PROGRESS: at 52.28% examples, 147299 words/s, in_qsize 7, out_qsize 0
2020-11-15 16:24:08,961 : INFO : EPOCH 150 - PROGRESS: at 53.46% examples, 14733

50845
Wall time: 3h 55min 27s


In [5]:
%%time

vectorizer_idf = TfidfVectorizer(norm='l1', use_idf=True)
vectorizer_idf.fit(data[column_name].fillna(''))

idf = vectorizer_idf.idf_
idf = idf / idf.sum()
vocabulary = vectorizer_idf.get_feature_names()
word2idf = dict(zip(vocabulary, idf))

print(idf.shape, len(vocabulary))

(391125,) 391125
Wall time: 42.2 s


In [6]:
%%time

wv_vocab = model.wv.vocab.keys()
embeddings = []

for words in tqdm(documents):
    words = list(wv_vocab & words)
    if words:
        idfs = [word2idf[word] for word in words]
        emb = model.wv[words]
        emb = np.dot(emb.T, idfs)
    else:
        emb = np.full(model.vector_size, np.nan)
    embeddings += [emb]
    
    
embeddings = np.stack(embeddings)

with open(path_emb, 'wb') as file:
    np.save(file, embeddings)

with open(path_emb, 'rb') as file:
    embeddings = np.load(file)
    
print(embeddings.shape)


(126784, 300)
Wall time: 11min 28s


In [30]:
model = Word2Vec.load(str(path_w2v))

# model.wv.get_vector('king')
# model.wv.distance('king', 'queen')

# model.wv.most_similar('доктор', topn=7)

# res = model.wv.most_similar('вич', topn=7)
# print(f'Request: вич')
# for name, cos in res:
#     print(f'{cos:.3f} - {name}')

# model.wv.most_similar(positive=['лукашенко', 'россия'], negative=['беларусь'], topn=7)
# model.wv.most_similar(positive=['лукашенко', 'украина'], negative=['беларусь'], topn=7)
# model.wv.most_similar(positive=['лукашенко', 'сша'], negative=['беларусь'], topn=7)
# model.wv.most_similar(positive=['лукашенко', 'германия'], negative=['беларусь'], topn=7)

# model.wv.most_similar(positive=['минск', 'россия'], negative=['беларусь'], topn=7)
# model.wv.most_similar(positive=['минск', 'украина'], negative=['беларусь'], topn=7)
# model.wv.most_similar(positive=['минск', 'сша'], negative=['беларусь'], topn=7)
# model.wv.most_similar(positive=['минск', 'германия'], negative=['беларусь'], topn=7)
# model.wv.most_similar(positive=['минск', 'франция'], negative=['беларусь'], topn=7)
# model.wv.most_similar(positive=['минск', 'чехия'], negative=['беларусь'], topn=7)
# model.wv.most_similar(positive=['столица', 'франция'], negative=['страна'], topn=7)

# model.wv.most_similar(positive=['журналист', 'девушка'], negative=['мужчина'], topn=7)

# model.wv.most_similar(positive=['чайник', 'электромобиль'], negative=['автомобиль'], topn=7)
# model.wv.most_similar(positive=['самокат', 'электромобиль'], negative=['автомобиль'], topn=7)

2020-12-05 19:34:48,450 : INFO : loading Word2Vec object from ..\..\data\model\w2v\model_w2v_clean_tutby_126784.model
2020-12-05 19:34:48,597 : INFO : loading wv recursively from ..\..\data\model\w2v\model_w2v_clean_tutby_126784.model.wv.* with mmap=None
2020-12-05 19:34:48,598 : INFO : loading vectors from ..\..\data\model\w2v\model_w2v_clean_tutby_126784.model.wv.vectors.npy with mmap=None
2020-12-05 19:34:48,638 : INFO : setting ignored attribute vectors_norm to None
2020-12-05 19:34:48,640 : INFO : loading vocabulary recursively from ..\..\data\model\w2v\model_w2v_clean_tutby_126784.model.vocabulary.* with mmap=None
2020-12-05 19:34:48,641 : INFO : loading trainables recursively from ..\..\data\model\w2v\model_w2v_clean_tutby_126784.model.trainables.* with mmap=None
2020-12-05 19:34:48,642 : INFO : loading syn1neg from ..\..\data\model\w2v\model_w2v_clean_tutby_126784.model.trainables.syn1neg.npy with mmap=None
2020-12-05 19:34:48,687 : INFO : setting ignored attribute cum_table to